# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from decimal import Decimal, getcontext
getcontext().prec = 50


import mpmath
mpmath.mp.dps = 75

from jorbit.utils.doubledouble import DoubleDouble, dd_polyval


In [2]:
# ((((((((bp[6]*7.*IAS15_H[n]/9. + bp[5])*3.*IAS15_H[n]/4. + bp[4])*5.*IAS15_H[n]/7. + bp[3])*2.*IAS15_H[n]/3. + bp[2])*3.*IAS15_H[n]/5. + bp[1])*IAS15_H[n]/2. + bp[0])*IAS15_H[n]/3. + a0)*dt*IAS15_H[n]/2. + v0)*dt*IAS15_H[n]

In [3]:
# ((((((((b.p6[k0]*7.*h[n]/9. + b.p5[k0])*3.*h[n]/4. + b.p4[k0])*5.*h[n]/7. + b.p3[k0])*2.*h[n]/3. + b.p2[k0])*3.*h[n]/5. + b.p1[k0])*h[n]/2. + b.p0[k0])*h[n]/3. + a0[k0])*r->dt*h[n]/2. + v0[k0])*r->dt*

In [4]:
# (((((((
#     (b.p6[k0]*7.*h[n]/9. + b.p5[k0])*3.*h[n]/4. + b.p4[k0])*5.*h[n]/7. + b.p3[k0])*2.*h[n]/3. + b.p2[k0])*3.*h[n]/5. + b.p1[k0])*h[n]/2. + b.p0[k0])*h[n]/3. + a0[k0])*r->dt*h[n]/2. + v0[k0])*r->dt*

In [8]:
b_len = 8
b_x_denoms = (1+jnp.arange(1, b_len + 1, 1)) * (2+jnp.arange(1, b_len + 1, 1))
b_v_denoms = jnp.arange(2, b_len + 2, 1)

b_x_denoms, b_v_denoms

(Array([ 6, 12, 20, 30, 42, 56, 72, 90], dtype=int64),
 Array([2, 3, 4, 5, 6, 7, 8, 9], dtype=int64))

In [9]:
b_len = 7
b_x_denoms = (1+jnp.arange(1, b_len + 1, 1)) * (2+jnp.arange(1, b_len + 1, 1))
b_v_denoms = jnp.arange(2, b_len + 2, 1)

b_x_denoms, b_v_denoms

(Array([ 6, 12, 20, 30, 42, 56, 72], dtype=int64),
 Array([2, 3, 4, 5, 6, 7, 8], dtype=int64))

In [10]:
from jorbit.data.constants import IAS15_H
n = 4

a0 = 2.12
bp = jnp.array([54.1, 93.0, 0.21, 5.21, 75.3, 8.32, 90.73])
h = IAS15_H[n]
dt = 0.04
v0 = 7.0
x0 = 3.0

xcoeffs = jnp.zeros(b_len + 3)
xcoeffs = xcoeffs.at[3:].set(bp * dt * dt / b_x_denoms)
xcoeffs = xcoeffs.at[2].set(a0 * dt * dt / 2.0)
xcoeffs = xcoeffs.at[1].set(v0 * dt)
xcoeffs = xcoeffs.at[0].set(x0)
xcoeffs = xcoeffs[::-1]

vcoeffs = jnp.zeros(b_len + 2)
vcoeffs = vcoeffs.at[2:].set(bp * dt / b_v_denoms)
vcoeffs = vcoeffs.at[1].set(a0 * dt)
vcoeffs = vcoeffs.at[0].set(v0)
vcoeffs = vcoeffs[::-1]

new_x = jnp.polyval(xcoeffs, h)
new_v = jnp.polyval(vcoeffs, h)

old_x = ((((((((bp[6]*7.*IAS15_H[n]/9. + bp[5])*3.*IAS15_H[n]/4. + bp[4])*5.*IAS15_H[n]/7. + bp[3])*2.*IAS15_H[n]/3. + bp[2])*3.*IAS15_H[n]/5. + bp[1])*IAS15_H[n]/2. + bp[0])*IAS15_H[n]/3. + a0)*dt*IAS15_H[n]/2. + v0)*dt*IAS15_H[n] + x0
old_v = v0 + (((((((bp[6]*7.*IAS15_H[n]/8. + bp[5])*6.*IAS15_H[n]/7. + bp[4])*5.*IAS15_H[n]/6. + bp[3])*4.*IAS15_H[n]/5. + bp[2])*3.*IAS15_H[n]/4. + bp[1])*2.*IAS15_H[n]/3. + bp[0])*IAS15_H[n]/2. + a0)*dt*IAS15_H[n]

new_x - old_x, new_v - old_v

(Array(0., dtype=float64), Array(0., dtype=float64))

In [ ]:
b_x_denoms = DoubleDouble(b_x_denoms)
b_v_denoms = DoubleDouble(b_v_denoms)
bp = DoubleDouble(bp)
dt = DoubleDouble(dt)

xcoeffs = DoubleDouble(jnp.zeros(b_len + 3))
tmp = bp * dt * dt / b_x_denoms
xcoeffs.hi = xcoeffs.hi.at[3:].set(tmp.hi)
xcoeffs.lo = xcoeffs.lo.at[3:].set(tmp.lo)

Array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -6.05515638e-18,
       -9.39248679e-18,  2.01470784e-20,  6.30162589e-19, -3.28848060e-18,
        9.34807787e-20, -6.83475498e-18], dtype=float64)

In [3]:
b = initialize_ias15_helper(1)
# print(bp[0.hi)

x0 = DoubleDouble(jnp.array([1.0, 0.0, 0.0]))
v0 = DoubleDouble(jnp.array([0.0, 1.0, 0.0]))

a0 = acceleration_func(x0)

step(x0, v0, a0, b)

# print(bp[0.hi)

5.649233359601495e-08, -5.388037824036623e-25
5.649320835508429e-08, -1.3876614877682261e-24
8.747592999253804e-13, -2.238183808514221e-30
2.213840311442581e-18, 1.8152355289655439e-34
3.2412436268176264e-24, 1.3856550845946262e-40
3.8274007215499996e-30, 8.068751029066524e-47
0.0, 0.0
0.0, 0.0
0.0, 0.0
0.0, 0.0


(<jorbit.integrators.ias15_dd.IAS15Helper at 0x10d9510f0>,
 DoubleDouble(0.0, 0.0),
 DoubleDouble(0.0, 0.0))